##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Pesquisa semântica com vizinhos mais próximos aproximados e embeddings de texto


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/tf2_semantic_approximate_nearest_neighbors"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/hub/tutorials/tf2_semantic_approximate_nearest_neighbors.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/google/nnlm-en-dim128/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a>
</td>
</table>

Este tutorial ilustra como gerar embeddings de um modelo do [TensorFlow Hub](https://tfhub.dev) (TF Hub) para dados de entrada fornecidos e como construir um índice de vizinhos mais próximos aproximados (ANN, na sigla em inglês) usando os embeddings extraídos. Em seguida, o índice pode ser usado para correspondência de similaridade e obtenção em tempo real.

Ao trabalhar com um corpus de dados grande, não é eficiente fazer a correspondência exata processando todo o repositório para encontrar os itens mais similares dada uma consulta em tempo real. Portanto, usamos um algoritmo de correspondência de similaridade aproximada, o que acarreta uma pequena perda de exatidão na procura de correspondências exatas de vizinhos mais próximos em troca de um aumento considerável da velocidade.

Neste tutorial, mostramos um exemplo de pesquisa de texto em tempo real em um corpus de manchetes de notícias para encontrar as manchetes mais similares a uma consulta. Diferentemente da pesquisa por palavras-chave, essa pesquisa captura a similaridade semântica codificada no embedding de texto.

Veja quais são as etapas deste tutorial:

1. Baixar os dados de amostra
2. Gerar embeddings dos dados usando um modelo do TF Hub
3. Criar um índice de ANN para os embeddings
4. Usar o índice para correspondência de similaridade

Usamos o [Apache Beam](https://beam.apache.org/documentation/programming-guide/) para gerar os embeddings a partir do modelo do TF Hub. Também usamos a biblioteca [ANNOY](https://github.com/spotify/annoy) do Spotify para criar o índice de vizinhos mais próximos aproximados.

### Outros modelos

Para modelos que tenham a mesma arquitetura, mas que tenham sido treinados com um idioma diferente, confira [esta](https://tfhub.dev/google/collections/nnlm/1) coleção. [Aqui](https://tfhub.dev/s?module-type=text-embedding) você encontra todos os embeddings de texto hospedados atualmente em [tfhub.dev](https://tfhub.dev/). 

## Configuração

Instale as bibliotecas necessárias.

In [ ]:
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'  # For gaussian_random_matrix.
!pip install annoy

Importe as bibliotecas necessárias.

In [ ]:
import os
import sys
import pickle
from collections import namedtuple
from datetime import datetime
import numpy as np
import apache_beam as beam
from apache_beam.transforms import util
import tensorflow as tf
import tensorflow_hub as hub
import annoy
from sklearn.random_projection import gaussian_random_matrix

In [ ]:
print('TF version: {}'.format(tf.__version__))
print('TF-Hub version: {}'.format(hub.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

## 1. Baixe os dados de amostra

O dataset [A Million News Headlines](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/SYBGZL#) (um milhão de manchetes de notícias) contém manchetes publicadas ao longo de 15 anos, obtidas da respeitável Australian Broadcasting Corp (ABC). Esse dataset de notícias tem um registro histórico resumido de eventos notáveis no mundo do início de 2003 até o fim de 2017, com um foco mais granular na Austrália.

**Formato**: dados em duas colunas separados por tabulação: 1) data de publicação e 2) texto da manchete. Temos interesse somente no texto da manchete.


In [ ]:
!wget 'https://dataverse.harvard.edu/api/access/datafile/3450625?format=tab&gbrecs=true' -O raw.tsv
!wc -l raw.tsv
!head raw.tsv

Por questões de simplicidade, vamos manter somente o texto da manchete e remover a data de publicação.

In [ ]:
!rm -r corpus
!mkdir corpus

with open('corpus/text.txt', 'w') as out_file:
  with open('raw.tsv', 'r') as in_file:
    for line in in_file:
      headline = line.split('\t')[1].strip().strip('"')
      out_file.write(headline+"\n")

In [ ]:
!tail corpus/text.txt

## 2. Gere embeddings dos dados

Neste tutorial, usamos o [Neural Network Language Model (NNLM)](https://tfhub.dev/google/nnlm-en-dim128/2) para gerar embeddings dos dados de manchetes. Os embeddings de frases podem ser usados facilmente para computar a similaridade de significado no nível de frase. Executamos o processo de geração de embeddings usando o Apache Beam.

### Método de extração de embeddings

In [ ]:
embed_fn = None

def generate_embeddings(text, model_url, random_projection_matrix=None):
  # Beam will run this function in different processes that need to
  # import hub and load embed_fn (if not previously loaded)
  global embed_fn
  if embed_fn is None:
    embed_fn = hub.load(model_url)
  embedding = embed_fn(text).numpy()
  if random_projection_matrix is not None:
    embedding = embedding.dot(random_projection_matrix)
  return text, embedding


### Converta para o método tf.Example

In [ ]:
def to_tf_example(entries):
  examples = []

  text_list, embedding_list = entries
  for i in range(len(text_list)):
    text = text_list[i]
    embedding = embedding_list[i]

    features = {
        'text': tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[text.encode('utf-8')])),
        'embedding': tf.train.Feature(
            float_list=tf.train.FloatList(value=embedding.tolist()))
    }
  
    example = tf.train.Example(
        features=tf.train.Features(
            feature=features)).SerializeToString(deterministic=True)
  
    examples.append(example)
  
  return examples

### Pipeline do Beam

In [ ]:
def run_hub2emb(args):
  '''Runs the embedding generation pipeline'''

  options = beam.options.pipeline_options.PipelineOptions(**args)
  args = namedtuple("options", args.keys())(*args.values())

  with beam.Pipeline(args.runner, options=options) as pipeline:
    (
        pipeline
        | 'Read sentences from files' >> beam.io.ReadFromText(
            file_pattern=args.data_dir)
        | 'Batch elements' >> util.BatchElements(
            min_batch_size=args.batch_size, max_batch_size=args.batch_size)
        | 'Generate embeddings' >> beam.Map(
            generate_embeddings, args.model_url, args.random_projection_matrix)
        | 'Encode to tf example' >> beam.FlatMap(to_tf_example)
        | 'Write to TFRecords files' >> beam.io.WriteToTFRecord(
            file_path_prefix='{}/emb'.format(args.output_dir),
            file_name_suffix='.tfrecords')
    )

### Geração da matriz de pesos com projeção aleatória

A técnica [projeção aleatória](https://en.wikipedia.org/wiki/Random_projection) é simples, mas poderosa, e é usada para reduzir a dimensionalidade de um conjunto de pontos que estão no espaço euclidiano. Para ver uma base teórica, confira o [lema de Johnson-Lindenstrauss](https://en.wikipedia.org/wiki/Johnson%E2%80%93Lindenstrauss_lemma).

Com a redução da dimensionalidade dos embeddings com projeção aleatória, menos tempo é necessário para criar e consultar o índice de ANN.

Neste tutorial, usamos a [projeção aleatória gaussiana](https://en.wikipedia.org/wiki/Random_projection#Gaussian_random_projection) da biblioteca [Scikit-learn](https://scikit-learn.org/stable/modules/random_projection.html#gaussian-random-projection).

In [ ]:
def generate_random_projection_weights(original_dim, projected_dim):
  random_projection_matrix = None
  random_projection_matrix = gaussian_random_matrix(
      n_components=projected_dim, n_features=original_dim).T
  print("A Gaussian random weight matrix was creates with shape of {}".format(random_projection_matrix.shape))
  print('Storing random projection matrix to disk...')
  with open('random_projection_matrix', 'wb') as handle:
    pickle.dump(random_projection_matrix, 
                handle, protocol=pickle.HIGHEST_PROTOCOL)
        
  return random_projection_matrix

### Defina os parâmetros

Se você quiser criar um índice usando o espaço de embeddings original sem projeção aleatória, defina o parâmetro `projected_dim` como `None`. Atenção: isso deixará o passo de indexação mais lento para embeddings de dimensão alta.

In [ ]:
model_url = 'https://tfhub.dev/google/nnlm-en-dim128/2' #@param {type:"string"}
projected_dim = 64  #@param {type:"number"}

### Execute o pipeline

In [ ]:
import tempfile

output_dir = tempfile.mkdtemp()
original_dim = hub.load(model_url)(['']).shape[1]
random_projection_matrix = None

if projected_dim:
  random_projection_matrix = generate_random_projection_weights(
      original_dim, projected_dim)

args = {
    'job_name': 'hub2emb-{}'.format(datetime.utcnow().strftime('%y%m%d-%H%M%S')),
    'runner': 'DirectRunner',
    'batch_size': 1024,
    'data_dir': 'corpus/*.txt',
    'output_dir': output_dir,
    'model_url': model_url,
    'random_projection_matrix': random_projection_matrix,
}

print("Pipeline args are set.")
args

In [ ]:
print("Running pipeline...")
%time run_hub2emb(args)
print("Pipeline is done.")

In [ ]:
!ls {output_dir}

Leia alguns dos embeddings gerados...

In [ ]:
embed_file = os.path.join(output_dir, 'emb-00000-of-00001.tfrecords')
sample = 5

# Create a description of the features.
feature_description = {
    'text': tf.io.FixedLenFeature([], tf.string),
    'embedding': tf.io.FixedLenFeature([projected_dim], tf.float32)
}

def _parse_example(example):
  # Parse the input `tf.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example, feature_description)

dataset = tf.data.TFRecordDataset(embed_file)
for record in dataset.take(sample).map(_parse_example):
  print("{}: {}".format(record['text'].numpy().decode('utf-8'), record['embedding'].numpy()[:10]))


## 3. Crie o índice de ANN para os embeddings

O [ANNOY](https://github.com/spotify/annoy) (Approximate Nearest Neighbors Oh Yeah) é uma biblioteca do C++ com bindings Python para procurar pontos no espaço que estão próximos de um dado ponto de consulta. Ela também cria estruturas de dados grandes baseadas em arquivos e somente leitura, que são mapeadas na memória. Ela é criada e usada pelo [Spotify](https://www.spotify.com) para fazer recomendações de músicas. Se você tiver interesse, pode testar outras alternativas ao ANNOY, como [NGT](https://github.com/yahoojapan/NGT), [FAISS](https://github.com/facebookresearch/faiss), etc. 

In [ ]:
def build_index(embedding_files_pattern, index_filename, vector_length, 
    metric='angular', num_trees=100):
  '''Builds an ANNOY index'''

  annoy_index = annoy.AnnoyIndex(vector_length, metric=metric)
  # Mapping between the item and its identifier in the index
  mapping = {}

  embed_files = tf.io.gfile.glob(embedding_files_pattern)
  num_files = len(embed_files)
  print('Found {} embedding file(s).'.format(num_files))

  item_counter = 0
  for i, embed_file in enumerate(embed_files):
    print('Loading embeddings in file {} of {}...'.format(i+1, num_files))
    dataset = tf.data.TFRecordDataset(embed_file)
    for record in dataset.map(_parse_example):
      text = record['text'].numpy().decode("utf-8")
      embedding = record['embedding'].numpy()
      mapping[item_counter] = text
      annoy_index.add_item(item_counter, embedding)
      item_counter += 1
      if item_counter % 100000 == 0:
        print('{} items loaded to the index'.format(item_counter))

  print('A total of {} items added to the index'.format(item_counter))

  print('Building the index with {} trees...'.format(num_trees))
  annoy_index.build(n_trees=num_trees)
  print('Index is successfully built.')
  
  print('Saving index to disk...')
  annoy_index.save(index_filename)
  print('Index is saved to disk.')
  print("Index file size: {} GB".format(
    round(os.path.getsize(index_filename) / float(1024 ** 3), 2)))
  annoy_index.unload()

  print('Saving mapping to disk...')
  with open(index_filename + '.mapping', 'wb') as handle:
    pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('Mapping is saved to disk.')
  print("Mapping file size: {} MB".format(
    round(os.path.getsize(index_filename + '.mapping') / float(1024 ** 2), 2)))

In [ ]:
embedding_files = "{}/emb-*.tfrecords".format(output_dir)
embedding_dimension = projected_dim
index_filename = "index"

!rm {index_filename}
!rm {index_filename}.mapping

%time build_index(embedding_files, index_filename, embedding_dimension)

In [ ]:
!ls

## 4. Use o índice para correspondência de similaridade

Agora podemos usar o índice de ANN para encontrar manchetes que estejam semanticamente próximas a uma consulta de entrada.

### Carregue o índice e os arquivos de mapeamento

In [ ]:
index = annoy.AnnoyIndex(embedding_dimension)
index.load(index_filename, prefault=True)
print('Annoy index is loaded.')
with open(index_filename + '.mapping', 'rb') as handle:
  mapping = pickle.load(handle)
print('Mapping file is loaded.')


### Método de correspondência de similaridade

In [ ]:
def find_similar_items(embedding, num_matches=5):
  '''Finds similar items to a given embedding in the ANN index'''
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=False)
  items = [mapping[i] for i in ids]
  return items

### Extraia o embedding de uma data consulta

In [ ]:
# Load the TF-Hub model
print("Loading the TF-Hub model...")
%time embed_fn = hub.load(model_url)
print("TF-Hub model is loaded.")

random_projection_matrix = None
if os.path.exists('random_projection_matrix'):
  print("Loading random projection matrix...")
  with open('random_projection_matrix', 'rb') as handle:
    random_projection_matrix = pickle.load(handle)
  print('random projection matrix is loaded.')

def extract_embeddings(query):
  '''Generates the embedding for the query'''
  query_embedding =  embed_fn([query])[0].numpy()
  if random_projection_matrix is not None:
    query_embedding = query_embedding.dot(random_projection_matrix)
  return query_embedding


In [ ]:
extract_embeddings("Hello Machine Learning!")[:10]

### Informe uma consulta para encontrar os itens mais similares

In [ ]:
#@title { run: "auto" }
query = "confronting global challenges" #@param {type:"string"}

print("Generating embedding for the query...")
%time query_embedding = extract_embeddings(query)

print("")
print("Finding relevant items in the index...")
%time items = find_similar_items(query_embedding, 10)

print("")
print("Results:")
print("=========")
for item in items:
  print(item)

## Quer aprender mais?

Saiba mais sobre o TensorFlow em [tensorflow.org](https://www.tensorflow.org/) e confira a documentação da API do TF Hub em [tensorflow.org/hub](https://www.tensorflow.org/hub/). Veja os modelos do TensorFlow Hub disponíveis em [tfhub.dev](https://tfhub.dev/), incluindo outros módulos de embeddings de texto e de vetor de características de imagens.

Confira também o curso [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/), que é uma introdução prática e acelerada ao aprendizado de máquina do Google.